In [1]:
import pandas as pd
import numpy as np

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

def get_shrunk_covariance_matrix(x, shrink=None):
        """Computes a covariance matrix that is "shrunk" towards a structured estimator. Code
            borrows heavily from the MATLAB implementation available by the authors online.
        
        Parameters
        ----------
        x : N x N sample covariance matrix of stock returns
        shrink : given shrinkage intensity factor; if none, code calculates
        
        Returns
        -------
        tuple : pandas.DataFrame which contains the shrunk covariance matrix
                : float shrinkage intensity factor
        
        """
        if x is None:
            raise ValueError('No covariance matrix defined')

        if type(x) == pd.core.frame.DataFrame:
            cov = x.as_matrix()
        elif type(x) == np.ndarray:
            cov = x
        else:
            raise ValueError(
                'Covariance matrix passed must be numpy.ndarray or pandas.DataFrame')

        if shrink is not None:
            shrinkage = shrink

        index = x.index
        columns = x.columns

        [t, n] = np.shape(cov)
        meanx = cov.mean(axis=0)
        cov = cov - np.tile(meanx, (t, 1))

        sample = (1.0 / t) * np.dot(cov.T, cov)

        var = np.diag(sample)
        sqrtvar = np.sqrt(var)

        a = np.tile(sqrtvar, (n, 1))
        rho = (sum(sum(sample / (a * a.T))) - n) / (n * (n - 1))

        prior = rho * (a * a.T)
        prior[np.eye(t, n) == 1] = var

        # Frobenius-norm of matrix cov, sqrt(sum(diag(dot(cov.T, cov))))
        # have to research this
        c = np.linalg.norm(sample - prior, 'fro') ** 2
        y = cov ** 2.0
        p = np.dot((1.0 / t), sum(sum(np.dot(y.T, y)))) - \
            sum(sum(sample ** 2.0))
        rdiag = np.dot((1.0 / t), sum(sum(y ** 2.0))) - sum(var ** 2.0)
        v = np.dot((cov ** 3.0).T, cov) / t - ((var * sample).T)
        v[np.eye(t, n) == 1] = 0.0
        roff = sum(sum(v * (a / a.T)))
        r = rdiag + np.dot(rho, roff)

        # compute shrinkage constant
        k = (p - r) / c
        shrinkage = max(0.0, min(1.0, k / t))
        sigma = np.dot(shrinkage, prior) + np.dot((1 - shrinkage), sample)

        return pd.DataFrame(sigma, index=index, columns=columns), shrinkage

df = pd.read_csv('../input/stocks_JT.csv')

def c_date(x):
    d, m, y = x.split('/')
    return '{}-{}-{}'.format(y, m, d)

df.date = pd.to_datetime(df.date.apply(c_date))
df = df.sort_values('date')

First, make sure all the stocks have at least 11 years of data. 

In [2]:
df['stock_count'] = df.groupby('TICKER')['date'].transform('count')
df = df[df.stock_count >= 12 * 11]

Determine all the periods for calculation.

In [3]:
all_dates = list(map(str, df.date.dt.date.unique()))

hist = []

start = 0
step = 12
duration = 12 * 11

for i in range(round(len(all_dates) / 12) - 11):
    period = all_dates[start + i * step:duration + i * step]
    hist.append(period)

Retrieve data for each period.

In [4]:
%%time

in_sample_matrices = []
out_of_sample_matrices = []

for period in hist:

    start, end = period[0], period[-1]
    
    print('Calculating period {} to {}'.format(start, end))
    
    mask = (df['date'] >= start) & (df['date'] <= end)

    df_hist = df[mask]
    df_hist['stock_count'] = df_hist.groupby('TICKER')['date'].transform('count')
    df_hist = df_hist[df_hist.stock_count == 11 * 12]

    df_hist['ret'] = df_hist.groupby('TICKER')['PRC'].transform(pd.Series.pct_change)

    ret_df = df_hist.groupby('TICKER')['ret'].apply(lambda x: list(x))

    ret_df = pd.DataFrame(dict(ret_df))

    in_sample = ret_df[: 12 * 10]
    out_of_sample = ret_df[12 * 10 :]
    
    in_sample_matrices.append(in_sample)
    out_of_sample_matrices.append(out_of_sample)

    

Calculating period 1978-08-31 to 1989-07-31
Calculating period 1979-08-31 to 1990-07-31
Calculating period 1980-08-29 to 1991-07-31
Calculating period 1981-08-31 to 1992-07-31
Calculating period 1982-08-31 to 1993-07-30
Calculating period 1983-08-31 to 1994-07-29
Calculating period 1984-08-31 to 1995-07-31
Calculating period 1985-08-30 to 1996-07-31
Calculating period 1986-08-29 to 1997-07-31
Calculating period 1987-08-31 to 1998-07-31
Calculating period 1988-08-31 to 1999-07-30
Calculating period 1989-08-31 to 2000-07-31
Calculating period 1990-08-31 to 2001-07-31
Calculating period 1991-08-30 to 2002-07-31
Calculating period 1992-08-31 to 2003-07-31
Calculating period 1993-08-31 to 2004-07-30
Calculating period 1994-08-31 to 2005-07-29
Calculating period 1995-08-31 to 2006-07-31
Calculating period 1996-08-30 to 2007-07-31
Calculating period 1997-08-29 to 2008-07-31
Calculating period 1998-08-31 to 2009-07-31
Calculating period 1999-08-31 to 2010-07-30
Calculating period 2000-08-31 to

In [5]:
for m in in_sample_matrices:
    print(m.shape)

(120, 1742)
(120, 1659)
(120, 1667)
(120, 1738)
(120, 1755)
(120, 1829)
(120, 1896)
(120, 1868)
(120, 1949)
(120, 2002)
(120, 1956)
(120, 1875)
(120, 1812)
(120, 1782)
(120, 1893)
(120, 2069)
(120, 2200)
(120, 2172)
(120, 2262)
(120, 2286)
(120, 2399)
(120, 2496)
(120, 2642)
(120, 2678)
(120, 2675)
(120, 2687)
(120, 2757)
(120, 2788)
(120, 2869)


In [6]:
for m in out_of_sample_matrices:
    print(m.shape)

(12, 1742)
(12, 1659)
(12, 1667)
(12, 1738)
(12, 1755)
(12, 1829)
(12, 1896)
(12, 1868)
(12, 1949)
(12, 2002)
(12, 1956)
(12, 1875)
(12, 1812)
(12, 1782)
(12, 1893)
(12, 2069)
(12, 2200)
(12, 2172)
(12, 2262)
(12, 2286)
(12, 2399)
(12, 2496)
(12, 2642)
(12, 2678)
(12, 2675)
(12, 2687)
(12, 2757)
(12, 2788)
(12, 2869)


Look at some examples.

In [7]:
in_sample_matrices[10]

,AA,AAME,AAPL,ABBK,ABF,ABL,ABM,ABP,ABRI,ABS,ABT,ABX,ABY,AC,ACE,ACET,ACG,ACK,ACLE,ACMTA,ACP,ACS,ACU,ACX,ACXM,ADAC,ADBE,ADCT,ADI,ADLA,ADM,ADMS,ADP,ADPT,ADV,ADX,AE,AEN,AEP,AEPI,...,WPS,WRE,WRI,WRS,WSBC,WSC,WSCI,WST,WSTNA,WTBK,WTEC,WTHG,WTRS,WWW,WWY,WY,XETA,XICO,XIOX,XOMA,XON,XRAY,XRIT,XRX,XTR,Y,YELL,YFED,YOCM,YORK,Z,ZAPS,ZF,ZIF,ZIGO,ZION,ZITL,ZMX,ZNT,ZSEV
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.039409,-0.317073,0.084639,-0.068493,0.164179,-1.906977,0.061611,-0.019048,-2.000000,0.082143,0.034853,-0.028777,-0.014925,0.012658,0.037037,-0.050847,-0.011364,0.055970,-1.948276,0.064516,-0.037594,-0.005376,-2.000000,0.023256,0.000000,-0.093750,0.115385,0.130435,-0.041667,0.087248,0.054054,0.103448,-0.092857,-0.086957,-0.028986,0.000000,-0.071429,-0.055556,0.013761,-0.035714,...,0.005917,-0.018265,0.034483,-2.269231,0.022901,0.010830,-0.060606,-0.054545,0.032258,-0.034483,-0.183907,0.005714,-0.038462,0.093750,0.045627,-0.005128,-0.083333,0.014493,0.153846,-0.033333,-0.037634,0.000000,0.123457,0.067285,-0.003584,0.032143,0.135135,-0.015385,-0.159420,-0.058824,0.136709,-0.378049,0.075000,0.012500,-2.048780,0.010753,-2.105263,0.068627,-0.075949,0.053571
2,0.054502,-0.035714,-0.106936,0.000000,-0.083333,-0.038462,-0.022321,-0.058252,-1.842105,-0.046205,-0.005181,0.014815,0.015152,0.000000,-0.035714,0.053571,0.034483,0.060071,-0.036364,0.075758,0.015625,-0.010811,-0.018519,-0.227273,-0.063492,0.017241,-0.014368,-0.105769,-0.043478,-0.030864,0.089744,-0.031250,-0.062992,-0.047619,-0.164179,0.056452,0.076923,0.000000,0.013575,0.037037,...,0.070588,0.018605,-0.023810,-0.151515,-0.007463,0.021429,0.129032,0.105769,-0.046875,0.071429,0.028169,-0.034091,-2.000000,-0.047619,0.138182,0.005155,-0.168831,-0.128571,-0.066667,-0.103448,0.000000,-1.970588,-0.219780,0.030435,0.043165,-0.003460,0.063492,0.093750,0.000000,-0.020833,-0.055679,0.117647,-0.046512,0.000000,-1.930233,0.047872,-2.238095,0.082569,-0.047945,-1.983051
3,-0.047191,-0.074074,-0.025890,-0.058824,0.041958,0.000000,-0.059361,-0.025773,0.000000,-0.010381,-0.041667,0.014599,-0.029851,0.025000,-0.129630,0.050847,-0.011111,-0.120000,0.000000,-1.887324,-0.030769,-0.038251,-0.056604,-0.088235,0.000000,-0.016949,-0.481050,0.032258,0.000000,-0.025478,-0.058824,-0.080645,-0.050420,-0.050000,-0.017857,-0.099237,-0.142857,-0.029412,-0.035714,-0.023810,...,-0.016484,-0.004566,-0.014634,-0.250000,-1.962406,0.090909,-0.200000,0.008696,-0.065574,-0.200000,-0.095894,0.041176,0.020000,-0.130000,-0.054313,0.020513,-0.203125,-0.081967,-0.142857,-0.038462,-0.033520,-0.121212,0.126761,-0.071730,-0.051724,-0.034722,-0.018657,0.000000,-0.051724,0.170213,-0.028302,-0.070175,-0.012195,-0.024691,-0.050000,-0.076142,-0.153846,-1.826271,-0.071942,-2.017241
4,0.056604,0.080000,0.069767,-0.046875,0.167785,0.026667,0.072816,0.111111,-0.083333,0.059441,0.046196,-0.057554,0.000000,0.024390,0.063830,0.000000,-0.056180,0.060606,0.018868,-2.047619,-0.039683,-0.096591,-0.040000,0.032258,-0.186441,0.310345,0.101124,0.010417,0.113636,0.117647,0.043750,0.175439,0.123894,-0.026316,0.090909,0.000000,0.000000,0.000000,0.009259,0.243902,...,-0.033520,-0.229358,0.014851,0.000000,-2.015625,0.051282,0.250000,0.034483,0.105263,0.166667,-0.121212,0.028249,-2.058824,0.000000,-0.023649,0.010050,-0.254902,0.125000,-0.041667,0.060000,0.017341,0.068966,-0.100000,0.061364,0.090909,0.021583,-0.038023,-0.014286,0.018182,-0.127273,0.004854,0.132075,0.024691,-0.075949,-0.026316,-0.032967,-0.045455,-2.056410,0.085271,0.101695
5,0.154018,-0.074074,-0.062112,0.049180,0.097701,-1.964286,0.072398,0.057143,-0.022727,0.108911,0.018182,0.160305,0.061538,0.083333,0.080000,-0.032258,0.023810,-0.021429,0.185185,-0.060606,0.041322,0.119497,0.041667,0.031250,0.062500,0.039474

In [8]:
out_of_sample_matrices[10]

,AA,AAME,AAPL,ABBK,ABF,ABL,ABM,ABP,ABRI,ABS,ABT,ABX,ABY,AC,ACE,ACET,ACG,ACK,ACLE,ACMTA,ACP,ACS,ACU,ACX,ACXM,ADAC,ADBE,ADCT,ADI,ADLA,ADM,ADMS,ADP,ADPT,ADV,ADX,AE,AEN,AEP,AEPI,...,WPS,WRE,WRI,WRS,WSBC,WSC,WSCI,WST,WSTNA,WTBK,WTEC,WTHG,WTRS,WWW,WWY,WY,XETA,XICO,XIOX,XOMA,XON,XRAY,XRIT,XRX,XTR,Y,YELL,YFED,YOCM,YORK,Z,ZAPS,ZF,ZIF,ZIGO,ZION,ZITL,ZMX,ZNT,ZSEV
120,-0.136159,-0.125000,-0.099278,-0.324675,-0.183246,-0.175966,-0.135593,0.060606,-0.068376,0.053385,-0.075075,-0.206107,-0.367347,-0.192683,-0.039474,-0.106557,-0.244186,-0.221095,-0.240000,0.047619,-0.148148,-0.102857,-0.142857,-0.159468,-0.122951,-0.176606,-0.187621,-0.337687,-0.345930,-0.033275,-0.124088,-0.007576,-0.453184,-0.010753,-0.283372,-0.167453,-0.309211,-0.201550,0.053857,-0.159509,...,0.040541,-0.059041,0.021559,-0.191489,-0.068627,-0.222127,-1.834711,-0.058427,-0.409836,-2.019608,-0.088889,-0.099567,-1.941176,-0.350000,-0.135889,-0.105655,-0.276316,-0.142857,-1.979592,-0.450000,-0.068505,-0.162990,-0.166667,-0.168147,-0.048485,-0.091703,-0.253906,-0.116564,-0.227273,-0.202381,-0.377682,-0.219512,-0.237113,-0.243697,-0.197368,-0.216837,-0.571429,-0.088235,-0.118881,-0.060606
121,0.185804,0.028571,0.222445,-2.120192,-0.112179,-0.005208,0.147059,0.192857,-0.087156,0.070457,0.128247,0.538462,0.137097,0.114804,0.136986,-0.032110,0.123077,0.114583,-0.052632,-1.931818,-0.028986,-0.044586,-0.041667,-0.185771,0.236760,0.069638,0.321429,-0.047887,0.142222,0.134058,0.116667,0.064885,-0.315068,-0.173913,0.065359,0.048159,0.276190,0.009709,0.078729,0.226277,...,0.061224,0.015686,0.064935,-0.052632,0.168421,0.025461,-1.891089,0.097852,0.333333,-0.115385,0.365854,-0.038462,0.140625,0.029586,-0.019355,0.123128,0.109091,0.111111,0.273438,0.333333,0.079274,0.048316,0.142857,-0.034875,-0.209130,-0.100962,0.130890,-0.027778,0.176471,0.194030,-0.041379,-2.078125,0.182432,0.122222,-0.139344,0.063518,0.308642,-0.161290,0.060847,-1.907258
122,0.116197,-0.166667,-0.026230,-2.012876,0.353791,0.007853,-0.042735,-0.110778,0.035176,0.030023,0.082014,0.068750,0.070922,0.075881,0.000000,-0.113744,0.047945,0.158879,0.388889,0.000000,-0.104478,0.020000,-0.043478,0.116505,0.012594,0.234375,0.070270,0.088757,0.237354,-0.038339,-0.003731,-0.025090,0.340000,0.703947,0.018405,0.097297,-0.044776,0.115385,0.002561,0.023810,...,-0.069930,0.096525,0.083841,-0.138889,0.040541,0.154966,0.088889,0.032609,-0.125000,-1.994567,-0.053571,0.085000,-2.342466,0.201149,0.064967,0.109630,0.163934,0.200000,-2.079754,0.227273,0.014159,0.150838,-0.287500,0.151180,-0.100671,0.013369,0.189815,0.142857,0.025000,-0.262500,-0.028777,-1.927536,0.000000,0.039604,0.485714,0.300153,0.292453,0.019231,0.019950,-0.013333
123,-0.063091,0.066667,-0.139731,0.084746,0.138667,-0.015584,0.189732,0.198653,0.048544,0.023543,0.021277,-0.064327,0.000000,0.085642,-0.012048,0.101604,0.019608,0.074597,0.080000,-1.991870,0.350000,1.003268,-0.113636,-0.113043,-0.059701,-0.122363,0.205387,0.298913,0.028302,-0.071429,0.101124,0.044118,-0.119403,0.003861,0.123494,-0.002463,-0.109375,0.155172,-0.052363,-0.020349,...,0.015038,-0.014085,0.022504,-0.032258,-0.047619,0.043736,-0.040816,0.023158,0.095238,-1.983605,-0.037736,-0.101382,-1.959184,0.038278,0.088803,0.070761,-0.028169,0.375000,-2.015152,-0.037037,0.047120,0.043689,0.228070,0.101858,0.143284,0.014512,0.003891,-0.150000,-0.012195,0.033898,-0.066667,-2.015625,0.028571,0.009524,0.269231,-0.048292,-0.160584,0.000000,-0.046455,-2.153153
124,0.004209,0.218750,0.281800,-0.132812,0.351288,-0.134565,0.039400,0.056180,-0.222222,0.116101,0.020833,-0.025000,-0.013245,-0.044084,-0.073171,0.029126,-0.064103,-0.094747,-0.148148,-2.016393,-0.006173,0.174551,-0.076923,0.039216,0.312169,-0.231971,0.044693,0.163180,0.535168,0.309481,-0.068027,0.084507,-0.144068,0.080769,-0.206434,0.051852,-0.192982,0.257463,0.014825,0.032641,...,0.044444,0.064286,-0.017882,-0.033333,0.072727,0.007812,-0.106383,0.174897,-0.043478,-2.000000,0.215686,0.025641,-2.000000,-0.023041,0.016312,0.013716,0.007246,-0.318182,-2.074627,-0.01

Esimated the shrunken covariance matrix. 

In [9]:
from tqdm import tqdm_notebook

cov_hat = []

for cov in tqdm_notebook(in_sample_matrices):
    cov_estimated, _ = get_shrunk_covariance_matrix(cov.cov())
    cov_hat.append(cov_estimated)
